In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import utils

In [71]:
class opt():
    def __init__(self):
        self.testimg='sao.jpg'
        self.workers=2
        self.batchSize=4 #'input batch size') 배치사이즈
        self.imageSize=224 #the height / width of the input image to network') 이미지 사이즈 (정방)
        self.lr=0.0002 # 러닝레이트
        self.m=0.9
        #self.net='' # 트레이닝 처음 시작할때 이 주석을 해제함
        self.net='model/net_1.pth' # 트레이닝을 이어서 할때 이 주석을 해제함
        self.cuda=True # 쿠다 옵션 (CPU 트레이닝 일시에는 False)

In [72]:
opt = opt() # opt 클래스 오브젝트 생성

In [67]:
#트랜스폼 선언
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.ToPILImage('RGB'),
    transforms.ToTensor()
])

In [68]:
import torchvision.models as models

# 모델 불러오기
model = models.vgg16()
if opt.cuda:
    model.cuda()

resume_epoch=0
if opt.net != '':
    model.load_state_dict(torch.load(opt.net,map_location=lambda storage, location: storage)['state_dict'])
    resume_epoch = torch.load(opt.net)['epoch']
    
optimizer = optim.SGD(model.parameters(), lr=opt.lr, momentum=opt.m)

In [73]:
#single
img = utils.load_image(opt.testimg, opt.imageSize)
img = transform(img)
inputs = torch.FloatTensor(1, 3, opt.imageSize, opt.imageSize)
if torch.cuda.is_available():
    inputs = inputs.cuda()
print(inputs.size())
inputs = Variable(inputs)
inputs.data[0:] = img
outputs = model(inputs)
predictions = outputs.max(1, keepdim=True)[1]
print("inference's predictions {}".format(predictions))

torch.Size([1, 3, 224, 224])
inference's predictions Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]



In [74]:
classes = ['makoto', 'sao', 'shinchan', 'zibri']
print(classes[predictions[0].data[0]])

makoto
